<a href="https://colab.research.google.com/github/rodrigofardin/Reconhecimento-de-Padroes/blob/main/Submission_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/titanic/train.csv')
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [5]:
y = np.array(data['Survived'])

In [6]:
X = data.drop('Survived', axis=1)

In [7]:
X_train = pd.read_csv('/content/drive/MyDrive/Datasets/titanic/train.csv')
X_test = pd.read_csv('/content/drive/MyDrive/Datasets/titanic/test.csv')
X_train.columns.isin(X_test.columns)

array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [8]:
print("Passengers (Rows): ", len(X_train))
print("Features (Columns): ", len(X_train.columns))

Passengers (Rows):  891
Features (Columns):  12


In [9]:
X_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [10]:
print("Passengers (Rows): ", len(X_test.index))
print("Features (Columns): ", len(X_test.columns))

Passengers (Rows):  418
Features (Columns):  11


In [11]:
X_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [12]:
X_train.columns[~X_train.columns.isin(X_test.columns)]

Index(['Survived'], dtype='object')

In [63]:
# Seleciona caracteristicas que nao serao analisadas
indesejadas = ['Cabin', 'Name', 'Ticket']

In [64]:
class AtributosDesejados(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    self.colunasIndesejadas = indesejadas
    return self
  def transform(self, X, y=None):
    return X.drop(indesejadas, axis=1)

In [65]:
# filtra somente as caracteristicas que sao number
class AtributosNumericos(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    self.colunasNumericas = X.select_dtypes(include='number').columns
    return self
  def transform(self, X, y=None):
    return X[self.colunasNumericas]

# tratamento das caracteristicas numericas
pipenum = Pipeline([
    ('atributos_numericos', AtributosNumericos()),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


In [66]:
# filtra somente as caracteristicas que sao object
class AtributosCategoricos(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    self.colunasCategoricas = X.select_dtypes(include='object').columns
    return self
  def transform(self, X, y=None):
    return X[self.colunasCategoricas]

# tratamento das caracteristicas numericas
pipecat = Pipeline([
    ('atributos_categoricos', AtributosCategoricos()),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])


In [67]:
unecaracteristicas = FeatureUnion([
    ('pipenum', pipenum),
    ('pipecat', pipecat)
])


In [68]:
preproc = Pipeline([
    ('atributos_desejados', AtributosDesejados()),
    ('unecaracteristicas', unecaracteristicas)
])


In [69]:
pipetotal = Pipeline([
    ('preproc', preproc),
    ('arvore', DecisionTreeClassifier(max_depth=25))
])


In [70]:
pipetotal.fit(X, y)
ypred = pipetotal.predict(X_test)
# accuracy_score(y, ypred)

In [33]:
output = pd.DataFrame({'PassengerId': X_test['PassengerId'], 'Survived': ypred})
output.to_csv('/content/drive/MyDrive/Datasets/titanic/submission.csv', index=False)


In [71]:
import numpy as np
from sklearn.model_selection import cross_validate

scores = cross_validate(pipetotal, X, y)
scores['test_score'], np.mean(scores['test_score'])

(array([0.58659218, 0.78651685, 0.80337079, 0.73595506, 0.8258427 ]),
 0.7476555144058754)

In [72]:
pipetotal.fit(X, y)
ypred = pipetotal.predict(X_test)

In [38]:
from sklearn.model_selection import GridSearchCV

parametros = {
    'arvore__max_depth': [None] + list(range(1,25,2)),
    'preproc__unecaracteristicas__pipenum__imputer__strategy': ['mean', 'median', 'most_frequent']
}

modelo = GridSearchCV(pipetotal, param_grid=parametros)

scores = cross_validate(modelo, X, y)
scores, np.mean(scores['test_score'])


({'fit_time': array([5.33653474, 6.7048018 , 5.41512823, 5.47987103, 5.59859538]),
  'score_time': array([0.00642395, 0.00714469, 0.00716519, 0.00720906, 0.0071187 ]),
  'test_score': array([0.68715084, 0.81460674, 0.81460674, 0.78089888, 0.79775281])},
 0.7790032013056305)

In [39]:
modelo.fit(X, y)
ypred = pipetotal.predict(X_test)

In [40]:
output = pd.DataFrame({'PassengerId': X_test['PassengerId'], 'Survived': ypred})
output.to_csv('/content/drive/MyDrive/Datasets/titanic/submission2.csv', index=False)